<div align="center">
  <a href="http://www.sharif.edu/">
    <img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" alt="SUT Logo" width="140">
  </a>
  
  # Sharif University of Technology
  ### Electrical Engineering Department

  ## Signals and Systems
  #### *Final Project - Spring 2025*
</div>

---

<div align="center">
  <h1>
    <b>Object Tracker</b>
  </h1>
  <p>
    An object tracking system using YOLO for detection and various algorithms (KCF, CSRT, MOSSE) for tracking.
  </p>
</div>

<br>

| Professor                  |
| :-------------------------: |
| Dr. Mohammad Mehdi Mojahedian |

<br>

| Contributors              |
| :-----------------------: |
| **Amirreza Mousavi** |
| **Mahdi Falahi** |
| **Zahra Miladipour** |


## 1: Preparing The Materials


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO

1.1 : Calculating HOG ( return the hog of the image)
 

In [2]:
def hog (image):
    # w , h = image.shape()
    filter = cv2.HOGDescriptor()
    result = filter.compute(image)
    return result

1.2 : Checking The Scale (return the scale of the image in the current frame)

In [6]:
def scaled_check (image ,h, source_hog):
    w , h = image.shape()
    n = np.arange(0.9 , 1.2 , 0.05)
    source = np.fft(source_hog)
    h_fft = np.fft(h)
    x = h_fft * source
    scale = 1
    min = 500
    for coefficient in n :
        new_img = cv2.resize(image , coefficient * w , coefficient * h)
        new_fft = np.fft(new_img)
        new_result = new_fft * h_fft
        diff = new_result - x
        sum = np.sum(diff ** 2)
        if (sum < min ):
            min = sum
            scale = coefficient 
    return scale    

1.3 : Prediction The Next Frame's Center